# Preparação Dos Dados


In [1]:
import numpy as np
import pandas as pd
import os

# Leitura dos dados originais
# mostrar todas as colunas
pd.set_option('display.max_columns', None)
file_path = 'data/TUCURUI_DIARIA_1939_2016.xlsx'
data = pd.read_excel(file_path, header=None, names=['Vazao'])

# Função para calcular derivadas percentuais
def calcular_derivadas(data):
    data['V1'] = data['Vazao'].pct_change() * 100  # Primeira derivada
    data['V2'] = data['V1'].diff() # Segunda derivada
    return data



# Função para criar defasagens
def criar_defasagens(data, colunas, entradas):
    defasagens_list = []

    for col in colunas:
        for i in range(entradas, 0, -1):
            defasagens_list.append(data[col].shift(i).rename(f'Entrada_{col}_{i}'))
        defasagens_list.append(data[col])

    for i in range(1, 12):
        defasagens_list.append(data['Vazao'].shift(-i).rename(f'Saida_{i}'))
    defasagens = pd.concat(defasagens_list, axis=1)
    return defasagens

# **MODELO 1**

In [2]:
# Criar datasets com defasagem apenas na coluna 'Vazao'
data_15 = criar_defasagens(data.copy(), ['Vazao'], 15)
data_30 = criar_defasagens(data.copy(), ['Vazao'], 30)
data_45 = criar_defasagens(data.copy(), ['Vazao'], 45)

# Pasta onde os arquivos serão salvos

output_folder = 'data/modelo_1'

# Criar a pasta se ela não existir
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Caminhos para os arquivos Excel
file_path_15 = os.path.join(output_folder, 'data_15_dias.xlsx')
file_path_30 = os.path.join(output_folder, 'data_30_dias.xlsx')
file_path_45 = os.path.join(output_folder, 'data_45_dias.xlsx')

# Salvar os DataFrames em arquivos Excel
data_15.to_excel(file_path_15, index=False)
data_30.to_excel(file_path_30, index=False)
data_45.to_excel(file_path_45, index=False)

print("DataFrames salvos com sucesso!")

KeyboardInterrupt: 

# **MODELO 2**



In [ ]:
# Calcular derivadas
data_com_derivadas = calcular_derivadas(data)

# Criar datasets com defasagem nas colunas 'V2', 'V1' e 'Vazao'
data_derivadas_15 = criar_defasagens(data_com_derivadas, ['V2', 'V1','Vazao'], 15)
data_derivadas_30 = criar_defasagens(data_com_derivadas, ['V2', 'V1','Vazao'], 30)
data_derivadas_45 = criar_defasagens(data_com_derivadas, ['V2', 'V1','Vazao'], 45)

# Pasta onde os arquivos serão salvos, dentro de 'data'
output_folder = 'data/modelo_2'

# Criar a pasta 'modelo_2' dentro de 'data' se ela não existir
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Caminhos  para os arquivos Excel
file_path_derivadas_15 = os.path.join(output_folder, 'data_derivadas_15_dias.xlsx')
file_path_derivadas_30 = os.path.join(output_folder, 'data_derivadas_30_dias.xlsx')
file_path_derivadas_45 = os.path.join(output_folder, 'data_derivadas_45_dias.xlsx')

# Salvar os DataFrames em arquivos Excel
data_derivadas_15.to_excel(file_path_derivadas_15, index=False)
data_derivadas_30.to_excel(file_path_derivadas_30, index=False)
data_derivadas_45.to_excel(file_path_derivadas_45, index=False)

print("DataFrames salvos com sucesso!")

DataFrames salvos com sucesso!


In [4]:
data = data[['V2', 'V1', 'Vazao']]
data

,V2,V1,Vazao
0,NaN,NaN,2110
1,NaN,-0.995261,2089
2,-0.871661,-1.866922,2050
3,-0.084298,-1.951220,2010
4,1.005946,-0.945274,1991
...,...,...,...
17284,-1.364289,0.389105,5418
17285,-3.028455,-2.639350,5275
17286,-1.531266,-4.170616,5055
17287,-0.517811,-4.688427,4818


# **MODELO 3**



In [9]:
import numpy as np
import pandas as pd
import os

# Leitura dos dados originais
pd.set_option('display.max_columns', None)
file_path = 'data/TUCURUI_DIARIA_1939_2016.xlsx'
data = pd.read_excel(file_path, header=None, names=['Vazao'])

# Função para calcular derivadas percentuais
def calcular_derivadas(data):
    data['V1'] = data['Vazao'].pct_change() * 100  # Primeira derivada
    data['V2'] = data['V1'].diff() # Segunda derivada
    return data

# Função para criar colunas binárias
def criar_colunas_binarias(data):
    bins = pd.DataFrame(index=data.index)
    bins['Bin1'] = ((data['V1'] > 40) & (data['V2'] > 0)).astype(int)
    bins['Bin2'] = ((data['V1'] < -40) & (data['V2'] < 0)).astype(int)
    bins['Bin3'] = (~((bins['Bin1'] == 1) | (bins['Bin2'] == 1))).astype(int)
    return bins

# Função para criar defasagens
def criar_defasagens(data, colunas, entradas):
    defasagens_list = []

    for col in colunas:
        for i in range(entradas, 0, -1):
            defasagens_list.append(data[col].shift(i).rename(f'Entrada_{col}_{i}'))
        defasagens_list.append(data[col])

    for i in range(1, 13):  # 12 saídas para frente
        defasagens_list.append(data['Vazao'].shift(-i).rename(f'Saida_{i}'))
    
    defasagens = pd.concat(defasagens_list, axis=1)
    return defasagens

# Função para criar o dataset completo com as colunas binárias
def criar_dataset_completo(data, entradas):
    data_com_derivadas = calcular_derivadas(data)
    colunas_binarias = criar_colunas_binarias(data_com_derivadas)
    data_com_derivadas = pd.concat([data_com_derivadas, colunas_binarias], axis=1)

    # Criar defasagens para colunas binárias
    defasagens_bin1 = [data_com_derivadas['Bin1'].shift(i).rename(f'Bin1_{i}') for i in range(entradas, 0, -1)]
    defasagens_bin2 = [data_com_derivadas['Bin2'].shift(i).rename(f'Bin2_{i}') for i in range(entradas, 0, -1)]
    defasagens_bin3 = [data_com_derivadas['Bin3'].shift(i).rename(f'Bin3_{i}') for i in range(entradas, 0, -1)]

    # Concatenar defasagens para colunas binárias
    defasagens_bin1_df = pd.concat(defasagens_bin1, axis=1)
    defasagens_bin2_df = pd.concat(defasagens_bin2, axis=1)
    defasagens_bin3_df = pd.concat(defasagens_bin3, axis=1)

    # Criar defasagens para as outras colunas
    defasagens_outros = criar_defasagens(data_com_derivadas, ['Vazao'], entradas)

    # Concatenar tudo na ordem correta
    defasagens = pd.concat([defasagens_bin1_df, defasagens_bin2_df, defasagens_bin3_df, data_com_derivadas[['V2', 'V1']], defasagens_outros], axis=1)
    
    return defasagens

# Criar datasets com defasagem e colunas binárias
data_completa_15 = criar_dataset_completo(data.copy(), 15)
data_completa_30 = criar_dataset_completo(data.copy(), 30)
data_completa_45 = criar_dataset_completo(data.copy(), 45)

# Pasta onde os arquivos serão salvos
output_folder = 'data/modelo_3'

# Criar a pasta se ela não existir
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Caminhos para os arquivos Excel
file_path_completa_15 = os.path.join(output_folder, 'data_completa_15_dias.xlsx')
file_path_completa_30 = os.path.join(output_folder, 'data_completa_30_dias.xlsx')
file_path_completa_45 = os.path.join(output_folder, 'data_completa_45_dias.xlsx')

# Salvar os DataFrames em arquivos Excel
data_completa_15.to_excel(file_path_completa_15, index=False)
data_completa_30.to_excel(file_path_completa_30, index=False)
data_completa_45.to_excel(file_path_completa_45, index=False)

print("DataFrames salvos com sucesso!")



DataFrames salvos com sucesso!
